In [1]:
import numpy as np
from random import shuffle, seed

from enigma.machine import EnigmaMachine
from vigenere_utils import *
import pandas as pd
import shutil, os

import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.contrib import rnn, seq2seq

In [2]:
# setup machine according to specs from a daily key sheet:

machine = EnigmaMachine.from_key_sheet(
       rotors='II IV V',
       reflector='B',
       ring_settings=[1, 20, 11],
       plugboard_settings='AV BS CG DL FU HZ IN KM OW RX')

# set machine initial starting position
machine.set_display('WXC')

# decrypt the message key
msg_key = machine.process_text('KCH')

# decrypt the cipher text with the unencrypted message key
machine.set_display(msg_key)

ciphertext = 'NIBLFMYMLLUFWCASCSSNVHAZ'
plaintext = machine.process_text(ciphertext)

print(plaintext)

THEXRUSSIANSXAREXCOMINGX


In [3]:
logs_path = '/home/user/tensorflow_logs/enigma_LSTM_long_rms'
seq_length = 140
ALPHA  = ALPHA.upper()

In [4]:
def get_random_sequence(ALPHA, seq_length):
        seq = ''.join(np.random.choice(a=list(ALPHA), replace=True, size=seq_length))
        return seq

In [5]:
def one_hot_encode(sequence, n_unique=len(ALPHA)):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[ALPHA.index(value)] = 1
		encoding.append(vector)
	return np.array(encoding)

In [6]:
np.random.seed(1616)
original_sequences = [get_random_sequence(ALPHA, seq_length) for i in range(2**20)]
seed(1616)
shuffle(original_sequences)

In [ ]:
def enigmate(s):
    machine.set_display(msg_key)
    return machine.process_text(s)

In [8]:
#encrypted_sequences = [machine.process_text(s) for s in original_sequences]
encrypted_sequences = [enigmate(s) for s in original_sequences]

In [9]:
original_sequences_oh = [one_hot_encode(s) for s in original_sequences]
encrypted_sequences_oh = [one_hot_encode(s) for s in encrypted_sequences]
original_argmax = []
encrypted_argmax = []

In [10]:
for s in original_sequences_oh:
    t = []
    for c in s:
        t.extend([np.argmax(c)])
    original_argmax.append(t)

In [11]:
for s in encrypted_sequences_oh:
    t = []
    for c in s:
        t.extend([np.argmax(c)])
    encrypted_argmax.append(t)

In [12]:
# import h5py
# h5f = h5py.File('data_enigma_long.h5', 'w')
# h5f.create_dataset('original_argmax', data=original_argmax)
# h5f.create_dataset('encrypted_argmax', data=encrypted_argmax)
# h5f.close()

# import h5py
# h5f = h5py.File('data_enigma_long.h5','r')
# original_argmax = h5f['original_argmax'][:]
# encrypted_argmax = h5f['encrypted_argmax'][:]
# h5f.close()

In [13]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
updater = Updater(token="312721681:AAEjt4EGxYAiRiemThOoccWQXgEHVXxvHCE")
dispatcher = updater.dispatcher
updater.start_polling()

def enigma_LSTM(bot, update):
     bot.send_message(chat_id=update.message.chat_id, \
     	text="Epoch: "+"{:.0f}".format(j)+" - Batch: "+"{:.0f}".format(i)+" - loss: "+"{:f}".format(training_loss_))

lstm_handler = CommandHandler('enigma_LSTM_6', enigma_LSTM)
dispatcher.add_handler(lstm_handler)

In [14]:
batch_size = 500
#batch_size = 100
num_steps = seq_length
num_classes = len(ALPHA)
state_size = 3000
num_layers = 3
learning_rate = 0.001
tf.reset_default_graph()

In [15]:
x = tf.placeholder(tf.int32, [batch_size, num_steps], name='input_placeholder')
y = tf.placeholder(tf.int32, [batch_size, num_steps], name='labels_placeholder')
init_state = tf.zeros([batch_size, state_size])


with tf.variable_scope("forward_pass"):
    rnn_inputs = tf.one_hot(x, num_classes)
    inputs_bn = tf.contrib.layers.batch_norm(rnn_inputs, 
                                              center=True, scale=True, 
                                              is_training=True,
                                             decay=0.9,
                                             updates_collections=None,
                                             epsilon=1e-5,
                                              scope='bn')

    #cell = tf.contrib.rnn.BasicRNNCell(state_size)
    with tf.variable_scope("LSTM"):
        tf.set_random_seed(1616)
        cell = tf.contrib.rnn.LSTMCell(state_size, state_is_tuple=True)
        rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, inputs_bn, dtype=tf.float32)

In [16]:
with tf.variable_scope('softmax'):
    W = tf.get_variable('W', [state_size, num_classes], initializer=tf.contrib.layers.xavier_initializer(uniform=False, seed=1616))
    b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    mult = tf.contrib.layers.batch_norm(tf.matmul(tf.reshape(rnn_outputs, [-1, state_size]), W) + b, 
                                              center=True, scale=True, 
                                              is_training=True,
                                             decay=0.9,
                                             updates_collections=None,
                                             epsilon=1e-5,
                                              scope='bn')
    logits = tf.reshape(
            mult,
            [batch_size, num_steps, num_classes])
    predictions = tf.nn.softmax(tf.nn.relu(logits))


In [17]:
with tf.variable_scope("backprop"):
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    total_loss = tf.reduce_mean(losses)
    #train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)
    train_step = tf.train.RMSPropOptimizer(learning_rate).minimize(total_loss)

In [18]:
with tf.variable_scope("tensorboard_summaries"):
    tf.summary.scalar("error", total_loss)
    tf.summary.histogram("predictions", predictions)
    tf.summary.histogram("logits", logits)
    # Create a summary to monitor accuracy tensor
    # Create summaries to visualize weights
    for var in tf.trainable_variables():
        tf.summary.histogram(var.name, var)

    merged_summary_op = tf.summary.merge_all()

INFO:tensorflow:Summary name forward_pass/bn/beta:0 is illegal; using forward_pass/bn/beta_0 instead.
INFO:tensorflow:Summary name forward_pass/bn/gamma:0 is illegal; using forward_pass/bn/gamma_0 instead.
INFO:tensorflow:Summary name forward_pass/LSTM/rnn/lstm_cell/kernel:0 is illegal; using forward_pass/LSTM/rnn/lstm_cell/kernel_0 instead.
INFO:tensorflow:Summary name forward_pass/LSTM/rnn/lstm_cell/bias:0 is illegal; using forward_pass/LSTM/rnn/lstm_cell/bias_0 instead.
INFO:tensorflow:Summary name softmax/W:0 is illegal; using softmax/W_0 instead.
INFO:tensorflow:Summary name softmax/b:0 is illegal; using softmax/b_0 instead.
INFO:tensorflow:Summary name softmax/bn/beta:0 is illegal; using softmax/bn/beta_0 instead.
INFO:tensorflow:Summary name softmax/bn/gamma:0 is illegal; using softmax/bn/gamma_0 instead.


In [19]:
def generate_batch(encrypted, original, batch_size):
    n_batches = len(encrypted)/batch_size
    for n in range(int(n_batches)):
        if((n+1)*batch_size+1 < len(encrypted_argmax)):
            yield(encrypted_argmax[n*batch_size:(n+1)*batch_size],\
                 original_argmax[n*batch_size:(n+1)*batch_size])
        else:
            yield(encrypted_argmax[n*batch_size:], \
                 original_argmax[n*batch_size:])

def gen_epoch(encrypted, original, n_epochs, batch_size):
    for e in range(n_epochs):
        yield(generate_batch(encrypted, original, batch_size))

In [20]:
if(os.path.exists(logs_path)):
    print("Removing logs")
    shutil.rmtree(logs_path)

In [ ]:
if(os.path.exists(logs_path)):
    print("Removing logs")
    shutil.rmtree(logs_path)

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
#sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
#sess = tf.InteractiveSession()
sess = tf.Session()

sess.run(tf.global_variables_initializer())
summary_writer = tf.summary.FileWriter(logs_path,
                                            graph=tf.get_default_graph())

training_state = np.zeros((batch_size, state_size))

st=-1

In [ ]:
for j, gen in enumerate(gen_epoch(encrypted_argmax, original_argmax, 2, batch_size)):
    print("Epoch")
    for i, (X, Y) in enumerate(gen):
        st+=1
        tr_losses, training_loss_, training_state, _, p, summary = \
                            sess.run([losses,
                                      total_loss,
                                      final_state,
                                      train_step,
                                     logits,
                                     merged_summary_op],
                                          feed_dict={x:X, y:Y})
        summary_writer.add_summary(summary, st)
        if(i % 1 == 0):
            print("Epoch: "+"{:.0f}".format(j)+" - Batch: "+"{:.0f}".format(i)+" - loss: "+"{:f}".format(training_loss_))
            print("input:  "+''.join([ALPHA[i] for i in X[0]]))
            print("target: "+''.join([ALPHA[i] for i in Y[0]]))
            print("pred:   "+''.join([ALPHA[np.argmax(i)] for i in p[0]]))

Epoch
Epoch: 0 - Batch: 0 - loss: 3.727187
input:  TUPAPOSUGGPJGPZYOFLPGEIETLFKBLMTUBGIFJJHOGBIPEGUWUDSJHXTBIRVBMDJLURRLXRLUZOAHSJAGHSMZFAECZLLTQGNXOKOSJRCLLORXPQPOYRAMLCXTXNHDSQEEJZICGVKLMVH
target: NDFQOAYNRTFMMZQBKEIJSWFDMSTIAJELHAAKIREPBONPESWLIOBMLLAHSGSQLQBKEQUXQLPKVSPGXVGRBYVOGKCNDOIJFNFOATMZYUMUUTGDMJNIHKOLHEWVNNIAHGBWHAUAPIHMKVOT
pred:   ACMDMVVQGGGNNKNKVTTVWGWEAATTPQIIIIIIIZJZROBFZEGGLQLLJROKRPFFPFFJJJJJJOJYJOOYRYYYNKWKKUUUUGGLAAIYYVVVVNJCCYVVVVVYVVXYCCCVCVYYPYYLLELEEGHNSTNR
Epoch: 0 - Batch: 1 - loss: 3.718301
input:  JCOWYQUWOXZQDFQEADKPRENXNRFCYZHVEOSMSYHYCQDDTUKFLXPTKSFTPNUJSQIVFHQJQVYWMXXHNGNSXUFAHGUIMCWJISXITACZMNBTACBBQTBTYABTHYLGAGCWZKRDSSDXHVEOEXTF
target: XKNPVWIIMYIKJGZSIYQJYWLZCBTSUXWWDHCUYVMGSCCEDOUCZVQXYZTHGUJBEMHGTLJKLULHBNNCFPMYHZNBEMEOSAOLZLYKSLYORLGXHQNCBHYBQFTHMTGCXTRBONCVGTXCZZXLILFR
pred:   JHHHXXXXXVBBBUBAYYSSSLFLFFUUMPRTARRRWWRKKWSSDDSTQOVVVVQUVQQQQYWJMIMWWMMXXXXRMIMYYKMUIIIIIIIZZJVEKKGGIDPIUUUPPPPPPPPPAPPOIIGXOVXSDJDOOJGREOOU
Epoch: 0 - Batch: 2 - loss

Epoch: 0 - Batch: 17 - loss: 3.662606
input:  LWWHFJKVRAPMYPDENMIECSCURFVAMUXWTAXJWYJECBFXVVZESLKAANOMZOAURMAZONRMJPIZBGYVXIWZJVBZHTBXMBZBXCCALSOMPZUDXSKTBXLQYJLNKBWVGXDVUQCUKRMTMKNISUWB
target: QPRYRCPLGLFJFZJSZILDLIPWSNSVGMQVQBBTPVEXSNYSKRVAPBJIXRVVIFWSTQMHUYUSKNTVMLHXHTVMSBAHERPTSXJDEXJEIUNKVXFYZOYKQZSYQMHBTICXPNUTTARNWOKQYQCVVKMZ
pred:   QQLOUZZSSSSZXZSEEEEEEGGCGHHHICIXIIUNHXHEAHUUUHPFGOQYYYXCCBCCXCCCCCXCCZEZZJJJNTZZZNHNRIROIIBBOUUUIYYCYGCCOYCCCOOYBBYYQPQQQQPPIIXXRXRCCCDZDDZZ
Epoch: 0 - Batch: 18 - loss: 3.656307
input:  ADUUHNYTXBWHUBHBRALBPBHMRAMAVNYNELVCAHANQLPEXMCZKUBAACLNPALRXLEVBDEICKGVOGSOZZJQMGSMPETCMPCWKXXAZBOMMFVKNAUVAIUJIKVSAUKGFETNZVTFLSUJXPLOWHBG
target: EUDKXXSRSMRBCMTYTGIAUTJOSYGVOBAADZFQRZVZWXEDUYTIFODIXMBCGCHCPXLGCXFEWYSZJLLNDKGWQNVOQZJLSUVSLCYEJRNKRIOPDPMGWOAEVYFOUJVCMKGDOBJKMTZGVROLBZRJ
pred:   YSDCRDDMOPOORRRRRRRPPPRRRUZDDDXXXLPQDDDDDLLLLLLLLCBCCCCDDYYLLLLSFSFFGSENENGONNNNCNWKWGWGCGGZSVVUBBBBCIIIQIINIFINENNNNNNNNEGAPPFIIQQQQQQQQOOO
Epoch: 0 - Batch: 19 - loss: 3

Epoch: 0 - Batch: 34 - loss: 3.511213
input:  ZDLARZUGWXKGOQVREEJHGZUKHYUKSWUDDTVEJHSMYRNBUYBRWWLXPNWEDSAXSRRURLMWFEIXRGGKNAERJFAZRQRAWQTOSWTRFPXYUIXKIQMSRQXTVJJORHIZFXVNMOWHGUNVODNSTJPG
target: VUZQFHIBPYQCZEOMQFGLSJEYPADIWQNGEKFHQZOVUFBFXMENIIGTZRQGYLWKEAONZHXFUCTJWLWDFQUASKBHTPZUUKDUYTUFCJRFKFDPTCUEVYMBIMNSBWASMNADWHAZSGRZDYCZPACJ
pred:   BBOYUOOOOOVOVBNBSGEWWOORRTTTVVRLDLDPPDYRRRXRXXJXXZQOQQQQYYYUYLUOOORYUYZUUHIVHHLLHHHHUBUUUBUAYLLLUUUUQMVVVVZVVVVCIGHHHAEEZUUUIRZZIIXIRBPLLJYJ
Epoch: 0 - Batch: 35 - loss: 3.495871
input:  MYZZGQULGRVTRYHIMXTEDUNLGAYBQQVLKYARLFMIWFQSTYGXMDDJBZPFPEVPEJJRHEJEUZUNCZEXOJYZGDMOMDNJILIYBGGQPTRBQAWXXBOZOEUGKIRFZKQPAKVVUZCKDRCLEOBKEZCP
target: ISLCIWIVRJEYPRTCPUXDQRLHYYAHDWZTVMGFGQHBQRHXDMWHBHBVMSSAGPELSVPMSBQIFDAFASDVBUSMBPZMFUHRLDLKHVFZQOXTEMATZNGQYWAVWWOEXZNTXEATTWRFCOGNNAKMIHPA
pred:   RKBBOBBIIIJIFTTTKTKAUODDOIIOBBBIBXXXXIIAIIZYYWWOAKKBBBJUQQJQQQEJZWEEERRRDRLORHEBHBEBEBBBEEEETJJMMKMTBBBUUUBBBPCOBMTTTTBZYWSSINNNNSSSSSOSSOLS
Epoch: 0 - Batch: 36 - loss: 3

Epoch: 0 - Batch: 51 - loss: 3.268695
input:  OVPPEMGOFQBYEOLOTCJEJUPILCPQQWIOUURQFGCQJTVRIDMTNCWDZDNAVUXRLBHRIUQZVCSCNTGVEHGZDKOPLOSFBBYUCMUPIBVCFORRBAEUMARFMTMBYENWMCWIIXFDNNLBETJKMMYD
target: WTFWSUZFZUSTAHWZRSGDZRCVXWQODQDRHXYCIEFJIPOKGBYOERUPPQUFWNZCBOIMMQJUPEGTECWXJEJMZFPCBSIMOXFOGBTCLRQAHEMQMPJLXCPMBUENWLQFFBYUXTPVTHTUNSSMYVLN
pred:   BSSWSQJEEWNISSQBNAAQQQQEEQQNLLVZWKFLCIILACPAAYEFHXQYQHQUBROOOOOOMMNNJJYYRRJJJJJJJJVWQBYTTTMKKKKDKTJJTRZZTCHQIIOCICIIIQQQQQQZZAUUUXXQQJPQBBKB
Epoch: 0 - Batch: 52 - loss: 3.256620
input:  RYFOCFTHGLEPRAUCWIRFPJDHWPIOWQJPOWPQVGMYJLMBUMNZVJEPSZLEHGJXGNKJLFPNHVKKFEMZPTCXPSECTQHQKKUJSJIJMWBCKGXJTYKGHVCJPUHHWRHXRZAISBOHQMDMYJSZVWKF
target: FSPEZRCCRAVDPCSIGMBGUZTLZURQSWGJPQTCKEHGIXAFXYJIHZSDRSBGJIUKWCUKETSHZUBLPMBWGIZNFOGPRPNGGQPLYDABWVTAUYDGIGYVSDFESTLTYQMVUDVUAVEZFFXHMUJSSQGR
pred:   FSMMATRPPIJQFOXDQFFHZZHZZZWZVLZWRVSLWWIIKQIIKIQQJQJQYQQJJJWNMTMTQTTQTWWWUUFQRFFNNYJJFLLLWWWBEEEEEEVFQIAAKKVIIBITGKDDZFZUFFCPYYRZLKKKKEGGJHQU
Epoch: 0 - Batch: 53 - loss: 3

Epoch: 0 - Batch: 68 - loss: 3.190360
input:  NHHFWHFKZTLIYIDUOSLEDMPUNPJQXTVQDHIMEKPQJXSHUZIXQBLVWLRNQNNGUNRASBTMAQNPNBHZZOBRALIJRLCQOYHMHXYRFBRVHQSXJGBGHYGQAUDQMVZYXWFSATLNLXUGEOKSGPEF
target: TITBTZXJFGYNFLJXKCIDQKCWCULOTHZBEOMUHIYJILIGXQSHCLGJUHHCUUPHKCOLHEKSMMJGEIYWDNOAKWJQTBAGBMGIBCXFCRXWFKCTCYNVSQHYGTQGHDBQASOXSXZUMYZJNABZNEQR
pred:   XZZTPZTUVPXGSKKKZYXQYYGJXXTELCJZYZFFJVQWENYLERVLLMNSSIFXCXXLKXFCYONKRNLGQQTOVVVMMNMMMNANZSZFZLSFTTMOZZYLEPMVLSSLXKKLKBOSOVUYYCXXXQNSJSVYYTJM
Epoch: 0 - Batch: 69 - loss: 3.189159
input:  TKNERIPWUMFOYEGQKBVLFTICSIVZLTUNROXYALUVOTMBCHMJNPPDXTRJMNBGBDDCIMVOYYUPUAVRPDPDQMKJXLSWBBJTUTXIIENROAOSFNHFHMNMHDXJZVMYFBHPUQXUDOKDPIFESWQZ
target: NCOOFBKIEBPZFQNLOJUHWBFJRVSERHNAZHBSRWDMBPAFNAYGEQQPAPHDAUMHLZBTMGBGBKAGVOJQGMTOMIYQSBIVOXZPWWYKLMOSLMVLGBRJSLQFOQYIXDHQMCLETANNCRMYCGMNVQEB
pred:   FIXQUMGQKKRRSSSPYMYIICKFYLZOQHXXOZLSCIXZZFFYALFEEECCCCCCAEQMMYYQKKJZSSDDDXJZCCCCZEYEAQYYMMMCKNNLLSQFBXBYCCXRWFFFFGUUVGJSMMMWXNNNZZVYGGTMYPPP
Epoch: 0 - Batch: 70 - loss: 3

...

Epoch: 0 - Batch: 935 - loss: 1.490280
input:  KNSIFSJKMBTRACNMMAFAYTYGZSVJKTHGETXGCEBJMEYWFYXCDNRDFLBGGKNUJSMPNWUICRIVWODDIRJERLGDGTOQPDSFUXNPBUCMLKGAYVLRBZKZIPPNDZXNHSADXQOPDLPHQASEUAVI
target: BXQDRYEJOMJLEAGRPGCBRBSPWLSUHHWDDKBONGGQEJFNQMPFRMZPIHLEPBPSMGAYWKREWITZRAEKUFGJUWEUZRXGRLBXWCECDSYKOQBFWJQDQXIOVCWBGKORBWVNIAEICEBMIOJNRJOQ
pred:   BXQDRYEJOMJLEAGRPGCBRBSPWLSUHHWDDKBONGGQVJFNBMPFRMZPIHOEPXPSMEAYWKRCAIRCZAKKKFEJAWNUNRNPQUOYWEEQDWABIYDPKWSMTXYXKSSBNONBMPUYNYZCYOCMEIGLHJZV
Epoch: 0 - Batch: 936 - loss: 1.572400
input:  GPSFHOOWXSEHVKJYEJTHTSKVMYRYOCWECTDZFGXHHDUBGKOCJPRQUGYYBKLZDLBDOUINENYZVBABWYVPTZNBOJUPSHPZZETLVLFKGYYOVSFEGJBXPPXHPWRVOPRCTOJVINLBPPLBCIOU
target: PWQBXAMISEVBTDDBQBXLAIZITAIDVPHUWKLDIENPRURFIIAFGQZEWUELSBHFHXDOUQCHIPLVUIRFLCWHYUFANCEYMWUVIAUGEAIMDGQVROISJPYHSCYTSHEXDJCFUHTDVHTUCRORXBVX
pred:   PWQBXAMISEVBTDDBQBXLAIZITAIDVPHUWKLDIENPRUDFIIAFGQZNWIELKXPFZEFGAQETIJTVRMCOBSSCRHHLZCWRYFQVXZBSJSIIDQQZJPCJYENZIINIIFUZLIBXGLAZADARWWARXADX
Epoch: 0 - Batch: 937 - loss

Epoch: 0 - Batch: 952 - loss: 1.535116
input:  DWGIIVNEFIRFSODBGLFDRMUHSRIVIDEMXBPQFMYOVVEYXZBRBYBTUIABFOOYRUECSOKETQJPOERCQDFIREUBNAOKFEARPDRBSCQJOAXEJZMHTITDVFWRPUIINIDSOQTFDMVZKJJVZPHN
target: CPJDGKLYZFWSKHJYWZCEYKELRBRAJGMHMATCIPPLZMPVUQENMADXWEXROFKWTFLTHJTINMNGJMAHMMDUUJLAOXXBHJKAQJPJTYVELMDNCEUMNOENIAPPSJAUTHUXZAJKCFSVGUSIAEZD
pred:   CPJDGKLYZFWSKHJYWZCEYKELRBRAJGMHMATCIPPLZMPVUVENMADXWEFROFTZAKLTPAJIIUOGNUATWZKUTHDLHCUAYARAABMTYAYWZCZJEXHMBPNNBMFEIRPPXPYKDAJMYFZHSAASHTZE
Epoch: 0 - Batch: 953 - loss: 1.371858
input:  LHPSNGHNWGGGJKGEERSPSBGFJLYRJFPJYRAUQQICYETAQQGVHIENMBNEAKYZAWAKYOVJPKWUDCRTKODICGRLEBRUIIBDESZSXNXXSGDKTAMDPFFEYRDCKSCOLNBSXTPZJCNRXHTSLRFE
target: QIFVQEWUPTOCDDNSQHOJGTAQVSANIOTPICGMJFZDUJGJFZWMVWSLBOUGMBIFNTMJPJBKVYZQHTAMTNFUINXRHLZALGSBXLVYAQRPYYXPIPURUNCJQEQXTNWLEXSXIXFHUQREVXAZKFTK
pred:   QIFVQEWUPTOCDDNSQHOJGTAQVSANIOTPICGMJFZDUJGJFZWMVWSLBORGMBIXDTRJPUBKVJZVHTWCDBMUTLCWJLTDMMLHXVVYEQEEYHGYBCRGSIIJQVNFUPFZQRNGNGIHUPDUCZGVMUMN
Epoch: 0 - Batch: 954 - loss

Epoch: 0 - Batch: 969 - loss: 1.313262
input:  UQUHBNUGCOSAOPXALYNPRYHRTQVFHECDXCCFSEOFQJSSWDXHAUQNJUBIXTVIADBFCBEMEOQHOHBPNOWBQWRKDNCSHPEAJMXEVACYOEVPPCOXTCWGRDSQOFFTELYQOQIBBWVSVVHNMECO
target: HBDYAXIBKCBXZZCTFDPJYCJNMESMKSFGMRSRYGSUWEIXYBPXOOPLLGLQKDENNZDBBEFSIWFWJPMJFNVKMLXVVHAZFUXRDBYAELYFLOOKSQGPNADVNQGGFADPLQWOZAMYOZSKXZVEYPPS
pred:   HBDYAXIBKCBXZZCTFDPJYCJNMESMKSFGMRSRYGSUWEIXYBPXOOPLLGLQUDENDZDZTEFXIRFWSDMJOBBOWQAVUFJVNQVRDBEAWEYFZAWAAYZZBADYVWLYZAAJJQQYFYPVVMBKBBMJFPPL
Epoch: 0 - Batch: 970 - loss: 1.358728
input:  IDKZFFQPSEZVCSMFIXAFCHCGCFPBVZTNISKAPRNPMUWDIGUMIABEDQWYXVMJEHKASVVEMDOJTTWARCMLPRSHRMJQMEEDIKROQSVKXVTILJSVNDCRLZPMZSUALGAFODFUPFDDSWDGFYJK
target: MUCCRRDDXSIEUWFHAUZGLNPPNNQHOXPAYJUXWJXHEDKEGPKVWYDQTDQLKWBBSPULHRBIAZHXYCGGVYNGFCVZTGTGSJXBZIPNPUQMACJBUVCGTVFLXOWWXNYDETVCZUPNZMXYUEQQHONE
pred:   MUCCRRDDXSIEUWFHAUZGLNPPNNQHOXPAYJUXWJXHEDKEGPKVWYDQTDQLKWBMSPJLHBYIAXHXCCZGFZZWDLONTSCNKANJKMORNROOZRBKSEPOBNFEEOIWWKRIOTIWLYWRCRYYVPYKRDAE
Epoch: 0 - Batch: 971 - loss

Epoch: 0 - Batch: 986 - loss: 1.348890
input:  MQFWIHZDHBLGNZTXCRPSGLXKYFGYBHLVYFOSMJTMXDCARPULJQPRTZPPPDBGARDGAQUICUCEJMSFSZRODJUIBFRGQSCKZKJBKUXJSXFVXZSINHQUEYRCXJEBXDDRZUDXNPWLSXJTXQIM
target: IBPPGZGPNMYCHPHUXHNYSOVYGNMDATOWIEEYDRRVAUDJZGKUGPQZDSSZGTMHNABVJUREWVVBOELBYKHDZELXLKZQZVVYIICJYSREYZUOZECYTTNSJKOXZURHAZUZODYSTIFNUHSFCWAL
pred:   IBPPGZGPNMYCHPHUXHNYSOVYGNMDATOWIEEYDRRVAUDJZGKUGPQZDSSZGTMHRAOVJJZEVVTMOBYRYMANUCDWLRPCPYGMJMWTMKZPHZHJZXPPBMYAJDNPNTRYCYYEHRYCXCFOGCAGCAAL
Epoch: 0 - Batch: 987 - loss: 1.222988
input:  LVOULYUYDNKOIEMHNVGPJTBVZVRRKEYTUGYZFBSCXYSMOHMHZTOFAIXRDMQNUCUDBNPIIDCIXFOJLNJGGWTEURKJULRAIZGZQHBCYEATNGIVJJDFVQXDQTSDXZTWTPGHFLCWFNJKKGTE
target: QTNKDSIEBPQZLQFFZKJJZBOIWIINHSALHIRDIDODAZICAAYXLGMHXEABYQGIKNKOCYSEEZVMLUPPWOGLBLCYITMRWDMRZPFQPFTASOWXDYFGGPWMIDYYEYPAADGBUIVZQEGFBBSMLTFK
pred:   QTNKDSIEBPQZLQFFZKJJZBOIWIINHSALHIRDIDODAZICAAYXLGMHXEABYQGIKNKOCYAEEXVTLPAPIFEPJECGDTFCWIPRLQCVPFMYSQXBLFFOEEWIOYZQYUPVNOJFJITZMMXFMSAEEKFK
Epoch: 0 - Batch: 988 - loss

Epoch: 0 - Batch: 1003 - loss: 1.224663
input:  OJYKTTVELXEOQDOKORPSKJEXNGWMYXTBDMGGMIFXIALMNWSBVIPUTJRPAWJXYGFPOKXPKUTYJBBMNPWJVUDNLPSXKMUXUPYUVSTHNXBNYBMRVJYVMDMAHIAWJHTXULJMGHQTTRRGJCAI
target: WZVUWPBYIYVZBKVNKHNYMZUZCZHFUZPQEYAODKCEJIXCCNIPHWQWDVHZMVUKZSZYUWMDJVIAOIMTFGVELZHWBQITGYPFWZXXEUBIZZGEWNUDRPBGBQELMBIFWIGSTMTGSNAQRPUQQDIQ
pred:   WZVUWPBYIYVZBKVNKHNYMZUZCZHFUZPQEYAODKCEJIXCCNIPHWQWDVHZMVUKZSZYUWMDJVIAOMMBOGBEWROOUOOOBSPEPUFHZYBFQZTLCTBMJVQJHWHUMYUYMMUOIATGTMGGGUUTTPLP
Epoch: 0 - Batch: 1004 - loss: 1.318766
input:  QYXVMUGULYAVJPWCOUDHMLETGTQZOBPUFOZEYYIVIIYLHTBHBKFSXTSBFMLUZRJEHDLUWCKUOXSLSZGJXCNIRDWLJAXHPRNNEHHNTQGXDHJXGBALTNUWUBFOJZNEIWPWVEAZJMXCCBID
target: LSISKMZNIXHEDZLIKXELKOUBYHPEVNTELHHHSVZMJAFQSJEXMFTMAPPROQHSYAPISXDZCEBQJNLIYKJEHRFXTUFCACENQOEOVFZLBKBTNFEPJGURUXAMAIDLWDIPXZFEILQVWFEXXIAN
pred:   LSISKMZNIXHEDZLIKXELKOUBYHPEVNTELHHHSVZMJAFQSJEXMFTMAPPROQHSYAPISXDFOVLVPNYXYMLERPHWZYUIDRELQOLLQSSLBEJZWSEZVNUQJQAKJVMZMHDKPFCFBPJHAFCPXRQY
Epoch: 0 - Batch: 1005 - l

In [23]:
saver = tf.train.Saver()

In [24]:
save_path = saver.save(sess, "/home/user/DL_pruebas/enigma_post_long_rms.ckpt")

In [ ]:
#saver.restore(sess, "/home/user/DL_pruebas/enigma_post_long_rms.ckpt")

In [ ]:
# summary_writer = tf.summary.FileWriter(logs_path,
#                                             graph=tf.get_default_graph())


In [25]:
def process_tweet(s, step_):
    import re
    s = re.sub(r'https.*', '', s)
    html_codes = (
            ("'", '&#39;'),
            ('"', '&quot;'),
            ('>', '&gt;'),
            ('<', '&lt;'),
            ('&', '&amp;')
        )
    for code in html_codes:
        s = s.replace(code[1], code[0])
    s = s.ljust(140)
    
    def preprocess_tweet(t):
        return re.sub(r'[^A-Z]', 'X', t.upper())
    
    tw = preprocess_tweet(s)
    #print(s)
    tw_arg = [np.argmax(one_hot_encode(enigmate(tw)), axis=1)]*batch_size
    #summary_op = tf.summary.text('tweet:', tf.convert_to_tensor('tweet:'+s))
    p_tw = sess.run(logits, feed_dict={x:tw_arg})
    
    #summary_writer.add_summary(tt, 1)
    
    P_tw = ''.join([ALPHA[np.argmax(i)] for i in p_tw[0]])
    
    l_pw = list(P_tw)
    for a in re.finditer(r'[^A-Z]', s.upper()):
        l_pw[a.span()[0]] = s.upper()[a.span()[0]]
    final = ''.join(l_pw)
    
    s = re.sub("…", "...", s)
    final = re.sub("…", "...", final)
    
    step_=step_+1
    text_pl = tf.placeholder(tf.string)
    summary_op = tf.summary.text("enigma_tweets", text_pl)
    
    s_ = sess.run(summary_op, feed_dict={text_pl:"prediction processed: "+final})
    summary_writer.add_summary(s_, step_)
    summary_writer.flush()
    s_ = sess.run(summary_op, feed_dict={text_pl:"prediction: "+P_tw})
    summary_writer.add_summary(s_, step_)
    summary_writer.flush()
    s_ = sess.run(summary_op, feed_dict={text_pl:"encrypted: "+enigmate(tw)})
    summary_writer.add_summary(s_, step_)
    summary_writer.flush()
    s_ = sess.run(summary_op, feed_dict={text_pl:"processed: "+tw})
    summary_writer.add_summary(s_, step_)
    summary_writer.flush()
    s_ = sess.run(summary_op, feed_dict={text_pl:"tweet: "+str(s)})
    summary_writer.add_summary(s_, step_)
    summary_writer.flush()

    return s, tw, enigmate(tw), P_tw, final

In [26]:
full_tweets = pd.read_json("/mnt/N/_Intercambio/NachoRosario/wwii_tweets.json")["text"]

In [27]:
step_=0

In [28]:
processed = [process_tweet(aa, step_) for aa in full_tweets]

In [ ]:
df = pd.DataFrame(processed)

In [ ]:
df.columns = ["tweet", "processed_tweet", "encrypted_tweet", "decryption_prediction", "decryption_processed"]

In [ ]:
df.to_csv(path_or_buf="/mnt/N/_Intercambio/NachoRosario/output.csv", index=False, sep="|")